In [111]:
import numpy as np
import math

# parameters
L = 0.5  # Length (before strain)
N = 50  # Spatial resolution
E = 207e9  # Young's modulus
r = 0.001  # Cross-section radius
rt1 = np.matrix([[0.02], [0], [0]])
rt2 = np.matrix([[-0.02], [0], [0]])
rho = 8000  # Density
g = [-9.81, 0, 0]  # Gravity
Bse = np.zeros(3)  # Material damping coefficients - shear and extension
Bbt = 1e-2 * np.eye(3)  # Material damping coefficients - bending and torsion
C = 0.1 * np.eye(3)
dt = 0.04  # Time step
alpha = -0.2  # BDF-alpha parameter
STEPS = 202  # Number of timesteps to completion
vstar = np.matrix([0, 0, 1])  # Value of v when static and absent loading
ustar = [0, 0, 0]  # Precurvature
vsstar = [0, 0, 0]
usstar = [0, 0, 0]
    
# Boundary Conditions
p = np.zeros((STEPS, N), dtype=object)
R = np.zeros((STEPS, N), dtype=object)
q = np.zeros((STEPS, N), dtype=object)
w = np.zeros((STEPS, N), dtype=object)
# global p, R, q, w
for i in range(STEPS):    # from 1 to STEPS
    p[i][0] = np.array([0, 0, 0])
    R[i][0] = np.eye(3)
    q[i][0] = np.array([0, 0, 0])
    w[i][0] = np.array([0, 0, 0])
    
nL = [0, 0, 0]
mL = [0, 0, 0]

# Dependent Parameter Calculations
A = math.pi*r**2    # Cross-sectional area
J = np.diag([math.pi*r**4, math.pi*r**4/4, math.pi*r**4/2])  # Inertia
G = 80e9    # shear modulus
Kse = np.matrix(np.diag([G*A, G*A, E*A]))   # Stiffness matrix - shear and extension
Kbt = np.diag([E*J[0, 0], E*J[1, 1], G*J[2, 2]])    # Stiffness matrix - bending and torsion
ds = L/(N-1)    # Grid distance (before strain)
c0 = (1.5 + alpha) / ( dt*(1+alpha) )   # BDF-alpha coefficients
c1 = -2 / dt
c2 = (0.5 + alpha) / (dt * (1 + alpha))
d1 = alpha / (1 + alpha)


In [137]:
def hat(y_in):
    y = np.matrix([[0, -y_in[2], y_in[1]],
                [y_in[2], 0, -y_in[0]],
                [y_in[1], y_in[0], 0]])
    return y

In [59]:
from scipy.optimize import fsolve

# Main Simulation
i = 1
# fsolve(staticIVP, np.zeros(6))
# applyStaticBDFalpha()

In [151]:
def dynamicODF(p, r, n, m, q, w):
    i=1

In [140]:
def staticODE(p, R, n, m, j):
    Rt = np.transpose(R)
    
    # v = Kse\ R' * n + vstar[ds*(j-1)]
    inv_Kse = np.linalg.inv(Kse)
    v = np.dot(inv_Kse, Rt * n + vstar)
    
    # u = Kbt\R'*m + ustar(ds*(j-1))      
    inv_Kbt = np.linalg.inv(Kbt + c0 * Bbt)
    u = np.dot(inv_Kbt, Rt*m + ustar)
    
    
    ptsb = np.matrix(hat(u)*rt1+v)
    print(ptsb)
    Tt = 0
    At = -Tt/np.linalg.norm(ptsb)**3*hat(ptsb)*hat(ptsb)
    B = hat(rt1)*At
    Gt = -At*hat(rt1)
    H =-B*hat(rt1)
    a = At*(hat(u)*ptsb)
    b = hat(rt1)*a

    fe=0*rho*A*g
    d = Kse*vsstar(ds*(j-1))-hat(u)*Kse*(v-vstar)-a-Rt*fe
    c = Kbt*usstar(ds*(j-1))-hat(u)*Kbt*(u-ustar)-hat(v)*Kse*(v-vstar(ds*(j-1)))-b


    Mat = [[Kse+At,Gt],[B,Kbt+H]]
    vs = 1/det(Mat)*((Kbt+H)*d-Gt*c)
    us = 1/det(Mat)*(-B*d+(Kse+At)*c)


    ps = R*v
    Rs = R*hat(u)
    ns = [0, 0, 0]
    ns = -fe
    ms = -hat(ps)*n
    
    return ps, Rs, ns, ms, vs, us, v, u

In [124]:
def staticIVP(G,i):
    n, m = np.zeros((3, 1), dtype=object), np.zeros((3, 1), dtype=object)
    n[i-1][0] = G[0:2]
    m[i-1][0] = G[3:5]
    
    # Euler's method
    for j in range (0, N-2):
        ps, Rs, ns, ms, vs, us, v, u = staticODE(p[i][j], R[i][j], n[i][j], m[i][j], j)
        p[i][j+1] = p[i][j] + ds * ps
        R[i][j+1] = R[i][j] + ds * Rs
        n[i][j+1] = n[i][j] + ds * ns
        m[i][j+1] = m[i][j] + ds * ms
    E = [n[i][N]-nL, m[i][N]-mL]
    print(E)
    return E

In [125]:
def find_tension():
    t = (i - 2) * dt

    P_desire = [0.25 * (1 - math.exp(-2 * t)), 0, 0]
    Pd_desire = [0.25 * (2 * math.exp(-2 * t)), 0, 0]
    Pdd_desire = [0.25 * (-4 * math.exp(-2 * t)), 0, 0]

    p_dot1 = R[i - 1][N - 1] * (hat(u[i - 1][N - 1]) * rt1 + v[i - 1][N - 1])
    p_ddot1 = R[i - 1][N - 1] * (hat(u[i - 1][N - 1]) * (hat(u[i - 1][N - 1]) * rt1 + v[i - 1][N - 1]) + hat(
    us[i - 1][N - 1]) * rt1 + vs[i - 1, N - 1])

    alph1 = (hat(p_dot1) * hat(p_dot1) * p_ddot1) / (norm(p_dot1) ^ 3)
    alph11 = (p_dot1) / (norm(p_dot1))

    alphM = alph1 + alph11

    ac = (ns + rho * A * g - R[i - 1][N - 1] * C * q[i - 1][N - 1] * abs(q[i - 1][N - 1])) / (rho * A)
    bc = -(alphM) / (rho * A)

    X_end = p[i - 1][N]
    Xd_end = (R[i - 1][N] * q[i - 1][N])

    alpa1 = 64.7
    alpa2 = 10

    Z1 = P_desire - X_end
    Z2 = Xd_end - Pd_desire - alpa1 * Z1

    bc_divide = Z1 - ac - alpa1 * (Z2 + alpa1 * Z1) - alpa2 * Z2 + Pdd_desire
    TT = np.linalg.lstsq(bc, bc_divide)

    if TT > 87:
        TT = 87  # fsolve can be solved in this condition

    e = P_desire - X_end
    e_dot = Pd_desire - Xd_end

    return TT

# Testing

In [147]:
G = [0.1490, 0, 0, 0, 0, 0]
print(i)
i=1
E = staticIVP(G,i)

1
[[array([[0.],
         [0.],
         [0.]]) array([[0.],
                       [0.],
                       [0.]]) array([[3.97887358e-06],
                                     [3.97887358e-06],
                                     [3.97887358e-06]])]
 [array([[0.],
         [0.],
         [0.]]) array([[0.],
                       [0.],
                       [0.]]) array([[3.97887358e-06],
                                     [3.97887358e-06],
                                     [3.97887358e-06]])]
 [array([[0.],
         [0.],
         [0.]]) array([[0.],
                       [0.],
                       [0.]]) array([[1.53772892e-06],
                                     [1.53772892e-06],
                                     [1.53772892e-06]])]]


ValueError: shapes (3,1) and (3,3) not aligned: 1 (dim 1) != 3 (dim 0)

In [ ]:
ptsb = np.array([[0], [0], [1]])
print (hat(ptsb))

print (hat(ptsb)*hat(ptsb))

In [146]:
Tt = 0
ptsb = np.array([[0], [0], [1]])
At = -Tt/np.linalg.norm(ptsb)**3*hat(ptsb)*hat(ptsb)
print(At)

[[array([[0.]]) array([[0.]]) array([[0.]])]
 [array([[0.]]) array([[0.]]) array([[0.]])]
 [array([[0.]]) array([[0.]]) array([[0.]])]]


In [ ]:
rt1 = [[0.2],[0],[0]]
